In [11]:
import xgboost
import lightgbm
import missingno
import sklearn

print(xgboost.__version__)
print(lightgbm.__version__)
print(missingno.__version__)
print(sklearn.__version__)

2.0.3
4.3.0
0.5.2
1.5.0


ID : 집을 구분하는 번호
date : 집을 구매한 날짜
price : 집의 가격(Target variable)
bedrooms : 침실의 수
bathrooms : 화장실의 수
sqft_living : 주거 공간의 평방 피트(면적)
sqft_lot : 부지의 평방 피트(면적)
floors : 집의 층 수
waterfront : 집의 전방에 강이 흐르는지 유무 (a.k.a. 리버뷰)
view : 집이 얼마나 좋아 보이는지의 정도
condition : 집의 전반적인 상태
grade : King County grading 시스템 기준으로 매긴 집의 등급
sqft_above : 지하실을 제외한 평방 피트(면적)
sqft_basement : 지하실의 평방 피트(면적)
yr_built : 지어진 년도
yr_renovated : 집을 재건축한 년도
zipcode : 우편번호
lat : 위도
long : 경도
sqft_living15 : 2015년 기준 주거 공간의 평방 피트(면적, 집을 재건축했다면, 변화가 있을 수 있음)
sqft_lot15 : 2015년 기준 부지의 평방 피트(면적, 집을 재건축했다면, 변화가 있을 수 있음)

In [12]:
import warnings

warnings.filterwarnings("ignore")

import os
from os.path import join

import pandas as pd
import numpy as np

import missingno as msno

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score
import xgboost as xgb
import lightgbm as lgb

import matplotlib.pyplot as plt
import seaborn as sns


# data_dir = os.getenv("HOME") + "/kaggle_kakr_housing/data"
data_dir = "./data"

# hint : os.getenv를 사용하거나 직접 경로를 작성

train_data_path = join(data_dir, "train.csv")
# 테스트, 즉 submission 시 사용할 데이터 경로
sub_data_path = join(data_dir, "test.csv")

print(train_data_path)
print(sub_data_path)


os.getenv("HOME")

data = pd.read_csv(train_data_path)
sub = pd.read_csv(sub_data_path)
print("train data dim : {}".format(data.shape))
print("sub data dim : {}".format(sub.shape))


y = data["price"]

del data["price"]
train_len = len(data)
data = pd.concat((data, sub), axis=0)
data.head()

./data\train.csv
./data\test.csv


FileNotFoundError: [Errno 2] No such file or directory: './data\\train.csv'

2. 간단한 전처리
각 변수들에 대해 결측 유무를 확인하고, 분포를 확인해보면서 간단하게 전처리를 하겠습니다.

결측치 확인
먼저 데이터에 결측치가 있는지를 확인하겠습니다.
missingno 라이브러리의 matrix 함수를 사용하면, 데이터의 결측 상태를 시각화를 통해 살펴볼 수 있습니다.